# ISCE Processing of SnowEx UAVSAR data
By: **Jack Tarricone** *(University of Nevada, Reno)* \
Guidance from: **Sam Neitlich** *(Montana State/JPL)*, **Yungling Lou** *(JPL)*, **Zach Keskenin** *(Boise State)*, **Ross Palomaki** *(Montana State)*, **Naheem Adebisi** *(Boise State)*

In [1]:
import os
import glob
import numpy as np
import isce
import shutil
from osgeo import gdal
import matplotlib.pyplot as plot
#import wget #(fix this)
#import uavsar_pytools
#import matplotlib.pyplot as plt
#import matplotlib
#import rasterio as rio
#from osgeo import gdal
#from rasterio.plot import show

This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able to obtain access to a version of the licensed sofware at
https://winsar.unavco.org/software/isce


ModuleNotFoundError: No module named '_gdal'

This tutorial is a recreation of the processing steps outlined [IsceUAVSARprocessing.ipynb](https://github.com/taliboliver/Isce_UAVSAR/blob/master/IsceUAVSARprocessing.ipynb), with addtional functionality added by the ```uavsar_pytools``` package. It is assumed that users have ISCE installed on thier local machines, which admittedly can be the most challenging step in this process.  For information on ISCE installoation please see the [Isce_UAVSAR](https://github.com/taliboliver/Isce_UAVSAR) by **Talib Oliver** *(JPL)* or Eric fieldings (find link from Zach)

In this notebook, we will interfermoterically process SLC stack from the Jemez Mountains, NM using the [Interferometric synthetic aperture radar Scientific Computing Environment (ISCE2)](https://github.com/isce-framework/isce2). We will then phase unwrap using [SNAPHU](https://web.stanford.edu/group/radar/softwareandlinks/sw/snaphu/), and geolocate the SLC data using ```geolocate_uavsar()```.

Past tutorials have assumed that your Jupyter environment will natively run command line code. In this notebook will run ISCE using python functionality.

## Create folder to work out of
First we'll create a new folder in our home directory to download files and run ISCE from

In [ ]:
# get home dir
home = '/Users/jacktarricone/'
os.chdir(home)
os.getcwd()

In [ ]:
# working folder name
isce_directory = 'jemez_isce'
    
# path to folder
working_dir = os.path.join(home,isce_directory)

In [ ]:
# make new folder
#os.mkdir(working_dir)
print("Directory '%s' created" %isce_directory)

In [ ]:
# nativate to new folder
os.chdir(working_dir)
os.getcwd()

In [ ]:
## create subdirectories

# download
dl = 'download'
download_path = os.path.join(working_dir,dl)
#os.mkdir(download_path)

# slc
SLC = 'SLC'
slc_path = os.path.join(working_dir,SLC)
#os.mkdir(slc_path)

# baselines
bl = 'baselines'
baselines_path = os.path.join(working_dir,bl)
#os.mkdir(baselines_path)

# referenceShelve
rs = 'referenceShelve'
referenceShelve_path = os.path.join(working_dir,rs)
#os.mkdir(referenceShelve_path)

# dem
dem = 'DEM'
dem_path = os.path.join(working_dir,dem)
#os.mkdir(dem_path)

# igrams
igrams = 'igrams'
igrams_path = os.path.join(working_dir,igrams)
#os.mkdir(igrams_path)

In [ ]:
# check folders were created
for file in os.listdir(working_dir):
    d = os.path.join(working_dir, file)
    if os.path.isdir(d):
        print(d)

## Data Download

Data links can be found on the UAVSAR site [here](https://uavsar.jpl.nasa.gov/cgi-bin/product.pl?jobName=alamos_35915_01#data"). In this case we'll be using SLCs from Feb 12th and 26th, both the LLH and LKV files, 1x1 pixel spacing, and just the HH polarization. By selecting these options, the site generates the list of ```wget``` download commands below.

 <p align="left">
  <img width="600" src="imgs/uavsar_data_portal.png">
</p>     
<p style="text-align: center;">
    (Data download screen)
</p>

Here the links are formatted as a string and saved as an exeutable shell file (.sh). This can also be down outside of this notebook by copying the links directly from the UAVSAR data portal and saving into your local text editor.

In [ ]:
# nativate to download folder
os.chdir(download_path)
os.getcwd()

In [ ]:
# create .sh file from links

links = ['wget https://downloaduav2.jpl.nasa.gov/Release30/alamos_35915_01/alamos_35915_01_BU.dop',
         'wget https://downloaduav2.jpl.nasa.gov/Release30/alamos_35915_01/alamos_35915_01_BU_s1_2x8.llh',
         'wget https://downloaduav2.jpl.nasa.gov/Release30/alamos_35915_01/alamos_35915_01_BU_s1_2x8.lkv',
         'wget https://downloaduav2.jpl.nasa.gov/Release30/alamos_35915_01/alamos_35915_20005_003_200212_L090HH_01_BU_s1_1x1.slc',
         'wget https://downloaduav2.jpl.nasa.gov/Release30/alamos_35915_01/alamos_35915_20005_003_200212_L090HH_01_BU.ann',
         'wget https://downloaduav2.jpl.nasa.gov/Release30/alamos_35915_01/alamos_35915_20013_000_200226_L090HH_01_BU_s1_1x1.slc',
         'wget https://downloaduav2.jpl.nasa.gov/Release30/alamos_35915_01/alamos_35915_20013_000_200226_L090HH_01_BU.ann']

with open('downloads.sh', 'w') as f:
    for links in links:
        f.write(links)
        f.write('\n')

In [ ]:
# ! sh downloads.sh

In [ ]:
# list files downloaded
for i in os.listdir(download_path):
    file = os.path.join(download_path, i)
    print(file)

## Prepare SLC Stack

In [ ]:
# nativate to download folder
os.chdir(working_dir)
os.getcwd()

In [ ]:
# copy doppler file to working directory
shutil.copy("/Users/jacktarricone/jemez_isce/download/alamos_35915_01_BU.dop", 
            "/Users/jacktarricone/jemez_isce/alamos_35915_01_BU.dop")

In [ ]:
# check function inputs
! prepareUAVSAR_coregStack_segmets.py -h

In this case our -i flag will the the download folder, the -d flag will be the doppler file, -0 will be the SLC directory.

In [ ]:
!prepareUAVSAR_coregStack_segmets.py -i download/ -d alamos_35915_01_BU.dop -o SLC

In [ ]:
# using gdal info the check everything worked
! gdalinfo /Users/jacktarricone/jemez_isce/SLC/20200212T1545/alamos_35915_20005_003_200212_L090HH_01_BU_s1_1x1.slc

In [ ]:
ds = gdal.Open("/Users/jacktarricone/jemez_isce/SLC/20200212T1545/alamos_35915_20005_003_200212_L090HH_01_BU_s1_1x1.slc", gdal.GA_ReadOnly)
# extract a part of the SLC to display
x0 = 0
y0 = 0
x_offset = 9789
y_offset = 69021  
slc = ds.GetRasterBand(1).ReadAsArray(x0, y0, x_offset, y_offset)
ds = None

fig = plt.figure(figsize=(14, 12))

# display amplitude of the slc
ax = fig.add_subplot(1,2,1)
ax.imshow(np.abs(slc), vmin = 0, vmax=1, cmap='gray')
ax.set_title("amplitude")

#display phase of the slc
ax = fig.add_subplot(1,2,2)
ax.imshow(np.angle(slc))
ax.set_title("phase")

plt.show()

slc = None
